In [1]:
import pandas as pd
import textdistance
import datetime
import os
from itertools import islice

In [1]:
# string similarity measure jarowinkler

def jarowinkler_slice(in_f, size, start, stop):
    out_f = "jarowinklernew"+str(start)+"-"+str(stop-1)+".csv"
    
    print(datetime.datetime.now()) # print waktu sekarang
    df_tokenized = pd.read_csv(in_f, delimiter=',', sep="\n") # read inputan data
    
    # some stuff to know the progress
    ch = 0 
    df_dummy = pd.DataFrame({'jarowinkler': [1]})
    df_dummy.to_csv("jarowinklernewtxt"+str(start)+"-"+str(stop-1)+"_"+str(ch)+".txt")
    
    df_slice = df_tokenized.iloc[start:stop , :] # men slice data (berbeda tiap kernel)
    nameslice = "slice "+str(start)+"-"+str(stop-1)+".csv" # nama untuk slice data yang akan dijadikan csv
    df_slice.to_csv(nameslice, header=['code','ingredient']) # menjadikan slice data menjadi csv
    
    reader = pd.read_csv(nameslice, delimiter=',', sep="\n", chunksize=size, index_col=0) # meload slice data, dengan CHUNKSIZE
    en=0
    for chunk in reader:
        err=''
        il=0
        df_lv = pd.DataFrame({'ingredient1': [],'ingredient2': [], 'jarowinkler': []}) 
        for index, row in chunk.iterrows():
            text1 = row['ingredient']
            for index2, row2 in islice(df_tokenized.iterrows(), index+1, None):
                text2 = row2['ingredient']
                try:
                    sim = textdistance.jaro_winkler.normalized_similarity(text1,text2)
                    il+=1
                    if sim>=0.8:
                        df_lv = df_lv.append({'ingredient1': row['ingredient'],'ingredient2': row2['ingredient'], 'jarowinkler': sim}, ignore_index=True)
                except Exception as e:
                    err = str(e)
                    en+=1
                    continue
        df_lv.to_csv(out_f, index=False, header=False, mode='a')
        renamebefore = "jarowinklernewtxt"+str(start)+"-"+str(stop-1)+"_"+str(ch)+".txt"
        ch+=1
        renameafter = "jarowinklernewtxt"+str(start)+"-"+str(stop-1)+"_"+str(ch)+".txt"
        os.rename(renamebefore, renameafter)
    print(datetime.datetime.now())
    print(err+str(en))

In [3]:
jarowinkler_slice("tokenized_nodup_tahap2.csv",4,1,8)

2019-06-29 07:07:00.471731


E:\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3191: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  if (yield from self.run_code(code, result)):


1
2
2019-06-29 07:12:26.639940
